In [1]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os
from dotenv import load_dotenv

load_dotenv()

vectorstore = FAISS.from_texts(
["Nitish worked at exa.ai"], embedding = OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
{"context":retriever, "question": RunnablePassthrough()}
| prompt | model | StrOutputParser()
)

chain.invoke("Where Nitish worked?")


'Nitish worked at exa.ai.'

In [2]:
template = """ Answert the question based only on the following context:
{context}
Question: {question}
Answer in the following language: {language}"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt
    |model
    |StrOutputParser()
)

In [3]:
chain.invoke({"question":"Where did Nitish work", "language": "Hindi!"})

'नितीश ने एक्सा.एआई में काम किया।'

## Conversational Retrieval Chain

In [4]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [21]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_template}")
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [doc.page_content for doc in docs]
    return document_separator.join(doc_strings)

In [22]:
_inputs = RunnableParallel(
    standalone_question = RunnablePassthrough.assign(
        chat_history = lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0.4)
    | StrOutputParser()
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x : x["standalone_question"]
}

conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [46]:
_inputs.input_schema.schema()

{'title': 'RunnableParallel<standalone_question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [19]:
retriever.invoke("Where Nitish works")

[Document(page_content='Nitish worked at exa.ai')]

In [24]:
conversational_qa_chain.invoke(
    {
        "question": "where did Nitish work?",
        "chat_history": [],
    }
)

AIMessage(content='Nitish worked at exa.ai.')

## With Memory and returning source documents

In [25]:
from langchain.memory import ConversationBufferMemory

In [26]:
memory = ConversationBufferMemory(
    return_messages = True, output_key="answer", input_key="question"
)

In [27]:
loaded_memory = RunnablePassthrough.assign(
chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    )
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
    "chat_history": lambda x: get_buffer_string(x["chat_history"])
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0.4)
    | StrOutputParser()
}

retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["question"]
}

final_inputs = {
    "context" : lambda x: _combine_documents(x["docs"]),
    "question" : lambda x: x["standalone_question"]
}

answer = {
"answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs")
}

final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [30]:
# inputs = {"question": "where did Nitish work?"}
# result = final_chain.invoke(inputs)
# result
loaded_memory.invoke({"question": "where did Nitish work?"})

{'question': 'where did Nitish work?', 'chat_history': []}

In [44]:
retrieved_documents

{'docs': RunnableLambda(itemgetter('standalone_question'))
 | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f83e66f0370>),
 'question': <function __main__.<lambda>(x)>}

In [43]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

inputs = {"question": "where did harrison work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='There is no information provided about where Harrison worked.'),
 'docs': [Document(page_content='Nitish worked at exa.ai')]}